# Model Training

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Constants
ANNOTATION_FILE = '/kaggle/input/fetus-dataset/Resized_ObjectDetection.xlsx'  
IMAGE_DIR = '/kaggle/input/fetus-dataset/Resized_images/Resized_images'  

# Load annotations for specific structures
annotations = pd.read_excel(ANNOTATION_FILE)
annotations = annotations[annotations['structure'].isin(['NT', 'nasal bone'])]

def load_and_preprocess_image(image_path):
    image = load_img(image_path, color_mode='rgb')  
    image = img_to_array(image)
    image = preprocess_input(image)
    return image

# Load and preprocess images
images = []
TARGET_STRUCTURES = ['NT', 'nasal bone']
annotations_list = []

# Unique filenames
unique_filenames = annotations['fname'].unique()

for fname in annotations['fname'].unique():
    image_path = os.path.join(IMAGE_DIR, f"{fname}.png")
    preprocessed_image = load_and_preprocess_image(image_path)
    images.append(preprocessed_image) 

    image_annotations = annotations[annotations['fname'] == fname]
    bbox = [0] * 8  

    for structure in target_structures:
        structure_annotations = image_annotations[image_annotations['structure'] == structure]
        if not structure_annotations.empty:
            
            h_min, w_min, h_max, w_max = structure_annotations.iloc[0][['h_min', 'w_min', 'h_max', 'w_max']]
            coords = [w_min, h_min, w_max, h_max]  
            if structure == 'NT':
                bbox[:4] = coords
            else:
                bbox[4:] = coords
                
    annotations_list.append(bbox)
    
# Convert to numpy arrays
images = np.array(images)
annotations_list = np.array(annotations_list)

# Split into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(images, annotations_list, test_size=0.2, random_state=42)

# Data Augmentation setup without flips
train_datagen = ImageDataGenerator(
    brightness_range=[0.8, 1.2],  
    channel_shift_range=0.1,  
    fill_mode='nearest'  
)

# Model Building - change the output layer to have 8 neurons
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

flat1 = Flatten()(base_model.output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(8, activation='linear')(class1)  
model = Model(inputs=base_model.inputs, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['accuracy'])  # Use Mean Squared Error

def reshape_annotations(annotations, num_features=2):  
    """Reshape annotations to match the model's output format."""
    reshaped_annotations = np.zeros((len(annotations), num_features * 4))
    for i, box_pair in enumerate(annotations):
        reshaped_annotations[i, :4] = box_pair[:4]  
        reshaped_annotations[i, 4:] = box_pair[4:]  
    return reshaped_annotations

# Prepare the annotations for the training and validation sets
y_train_reshaped = reshape_annotations(y_train, num_features=2)  
y_val_reshaped = reshape_annotations(y_val, num_features=2)

# Custom Training Loop
BATCH_SIZE = 150
EPOCHS = 10

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    batches = 0

    for x_batch in train_datagen.flow(x_train, batch_size=BATCH_SIZE, seed=epoch):
        start = batches * BATCH_SIZE
        end = (batches + 1) * BATCH_SIZE
        y_batch = y_train_reshaped[start:end]

        model.train_on_batch(x_batch, y_batch)

        batches += 1
        if batches >= len(x_train) / BATCH_SIZE:
            break

    # Evaluate on validation data
    val_loss, val_acc = model.evaluate(x_val, y_val_reshaped)
    print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")


# Save the model
model.save('fetal_model.h5')


Epoch 1/5
8/8 [==============================] - 57s 7s/step - loss: -2189802.0000 - accuracy: 0.1101
Validation loss: -2189802.0, Validation accuracy: 0.11013215780258179
Epoch 2/5


# Model Prediction

In [13]:
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Constants
MODEL_PATH = 'fetal_model.h5'  
IMAGE_PATH = '100.png' 
OUTPUT_IMAGE_PATH = 'annotated_image.png'  
OUTPUT_EXCEL_PATH = 'predicted_boxes.xlsx'  

# Load the trained model
model = load_model(MODEL_PATH)

# Load and preprocess an image
def preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size, color_mode='rgb')
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

# Predict bounding boxes for the new image
preprocessed_image = preprocess_image(IMAGE_PATH)
predicted_boxes = model.predict(preprocessed_image)[0]

# Draw bounding boxes on the original image
original_image = cv2.imread(IMAGE_PATH)
for i in range(0, len(predicted_boxes), 4):
    x_min, y_min, x_max, y_max = predicted_boxes[i:i+4].astype(int)
    cv2.rectangle(original_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

# Save the image with bounding boxes
cv2.imwrite(OUTPUT_IMAGE_PATH, original_image)

# Save the predicted bounding box coordinates to an Excel file
# Assuming the first four values are for 'NT' and the next four are for 'nasal bone'
df = pd.DataFrame([predicted_boxes], columns=['NT_x_min', 'NT_y_min', 'NT_x_max', 'NT_y_max', 'nasal_bone_x_min', 'nasal_bone_y_min', 'nasal_bone_x_max', 'nasal_bone_y_max'])
df.to_excel(OUTPUT_EXCEL_PATH, index=False)

print(f"Annotated image saved at {OUTPUT_IMAGE_PATH}")
print(f"Predicted bounding box coordinates saved at {OUTPUT_EXCEL_PATH}")


1/1 [==============================] - 1s 559ms/step
Annotated image saved at annotated_image.png
Predicted bounding box coordinates saved at predicted_boxes.xlsx
